In [1]:
# This code is for checking 
# Preprocessing method for 행정동

In [2]:
from glob import glob

import pandas as pd
import geopandas as gpd

In [3]:
def read_gdf(file_path, encoding):
    gdf = gpd.read_file(file_path, encoding=encoding)
    return gdf


def gdf2df(gdf):
    return pd.DataFrame(gdf)


read_gdf_with_cp949 = lambda file_path: read_gdf(file_path, "cp949")
read_shp = lambda file_path: gdf2df(read_gdf_with_cp949(file_path))

In [4]:
file_path = "data/raw_data/행정동_좌표/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"
hdong_shp_df = read_shp(file_path)
hdong_shp_df.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry
0,20230701,사직동,11010530,"POLYGON ((197702.069 553187.311, 197703.431 55..."
1,20230701,삼청동,11010540,"POLYGON ((198170.457 553770.678, 198172.189 55..."
2,20230701,부암동,11010550,"POLYGON ((196621.023 556395.880, 196628.323 55..."
3,20230701,평창동,11010560,"POLYGON ((197800.719 559064.245, 197782.581 55..."
4,20230701,한남동,11030740,"POLYGON ((200404.268 550031.323, 200404.019 55..."


In [5]:
def read_excel(file_path, params):
    return pd.read_excel(file_path, **params)


read_excel_params = {
    "index_col": None,
    "header": 1,
    "converters": {"시도코드": str, "시군구코드": str, "읍면동코드": str},
}

read_excel_with_params = lambda file_path: read_excel(file_path, read_excel_params)

In [6]:
file_path = "./data/raw_data/행정동_좌표/센서스 공간정보 지역 코드.xlsx"
hdong_shp_meta_df = read_excel_with_params(file_path)
hdong_shp_meta_df.head()

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭
0,11,서울특별시,010,종로구,530,사직동
1,11,서울특별시,010,종로구,540,삼청동
2,11,서울특별시,010,종로구,550,부암동
3,11,서울특별시,010,종로구,560,평창동
4,11,서울특별시,010,종로구,570,무악동


In [7]:
hdong_shp_meta_df["ADM_CD"] = (
    hdong_shp_meta_df["시도코드"] + hdong_shp_meta_df["시군구코드"] + hdong_shp_meta_df["읍면동코드"]
)
hdong_shp_meta_df["rough_name"] = hdong_shp_meta_df["시도명칭"] + "_" + hdong_shp_meta_df["시군구명칭"].fillna("")

In [8]:
preproc_hdong_shp_df = hdong_shp_df.merge(hdong_shp_meta_df.loc[:, ["ADM_CD", "rough_name"]], on=["ADM_CD"], how="left")

In [9]:
preproc_hdong_shp_df = preproc_hdong_shp_df.loc[:, ["ADM_NM", "ADM_CD", "rough_name", "geometry"]]
preproc_hdong_shp_df.columns = ["detail_name", "detail_code", "rough_name", "geometry"]

In [10]:
preproc_hdong_shp_df

,detail_name,detail_code,rough_name,geometry
0,사직동,11010530,서울특별시_종로구,"POLYGON ((197702.069 553187.311, 197703.431 55..."
1,삼청동,11010540,서울특별시_종로구,"POLYGON ((198170.457 553770.678, 198172.189 55..."
2,부암동,11010550,서울특별시_종로구,"POLYGON ((196621.023 556395.880, 196628.323 55..."
3,평창동,11010560,서울특별시_종로구,"POLYGON ((197800.719 559064.245, 197782.581 55..."
4,한남동,11030740,서울특별시_용산구,"POLYGON ((200404.268 550031.323, 200404.019 55..."
...,...,...,...,...
3523,운정6동,31200516,경기도_파주시,"POLYGON ((176367.906 568550.646, 176366.742 56..."
3524,운정5동,31200515,경기도_파주시,"POLYGON ((176896.770 570836.185, 176894.169 57..."
3525,새롬동,29010521,세종특별자치시_세종시,"POLYGON ((223193.497 431749.316, 223183.276 43..."
3526,도담동,29010514,세종특별자치시_세종시,"POLYGON ((224447.487 434259.018, 224447.454 43..."
